# 1. Introduction
## Data considerations

This project aims to build an algorithm to detect emotions from images with four different kinds of facial expressions: anger, disgust, happiness, and sadness. Before diving into the analysis, we would like to give an overview of how the data was collected: 

**1. Where do the data come from? (To which population will results generalize?)**
* The data comes from the expanded Cohn-Kanada (CK+) database, which is released to promote research on detecting facial expressions (Lucey, et al., 2010; p. 1). The databse contains 101 pictures of participants, where the age range is 18 to 50, and 69% female, 81%, Euro-American, 13% Afro-American, and 6% other groups (Lucey, et al., 2010; p. 1).
* As can be seen, the data is only generalizable to Euro/ American enthnicity, meaning other groups are not represented well by the data. Furthermore, the participants for the study were composed of actors or actresses, meaning they would fake their facial expressions to look more extreme than in real life. 

**2. What are candidate machine learning methods? (models? features?)**
* As the problem issues binary outcomes, we can classify this problem as a binary classification problem. The  candidates are:
<div style="display:flex; flex-direction: row; flex-wrap: nowrap; align-items: stretch; width:100%;">
    <div style="display:inline-block;width:45%;">
        <ul>
<h5> Model Candidates: </h5>          
<li> LDA
<li> QDA
<li> KNN 
<li> LASSO and RIDGE
<li> Support Vector Machines
<li> Tree and Random Forest
        </ul>
    </div>
    <div style="display:inline-block;width:45%;">
        <ul>
<h5> Feature Candidates: </h5>  
<li> Spectral Features from Histogram Analysis
<li> Edge Features Using Frey Slate Functions
<li> Histogram related gradients and other features
<li> Bag of features - which we did not explore
        </ul>
    </div>

**3. What is the Bayes' error bound? (Any guestimate from scientific literature or web resources?)**
* As always, we assum humans are good at recognizing facial expressions. Although in reality, some facial expressions could be very subtle. 
* To have an idea of a lower bound on the Bayes bound (i.e., the minimum accuracy that should be achievable). The best 'machine' we have at hand to recognize emotion from facial expression in the human brain. How often do human judges get it correct? In a study by Mollahosseini et al. (2018) an estimate for human classification inter-rater agreement was obtained for 11 emotions. For the four included in this data set they are:


| disgust  |  anger  |  happy  |  sad  |
|---------:|--------:|--------:|-------|
|  67.6%   | 62.3%   | 79.6%   | 69.7% |


Keep this in mind when evaluating the performance of the classifiers that you'll train.

As always, it's handy to evaluate how the algorithm does on the training set: If the training set is not classified accurately, how can you expect the test set to do any better. This obvious fact is often overlooked—surprisingly.

--------------------------------

# 2. Packages and importing data

In this step, we download all the necessary packages and import dataset for analysis.

In [ ]:
## Importing packages

library(tidyverse) # metapackage with lots of helpful functions
library(png) # package that can be used to read png image files in a simple format
library(caret)

## Reading in files

# You can access files the "../input/" directory.
# You can see the files by running  

list.files(path = "../input/")

In [ ]:
# Show the availabe directories
dirs = dir("../input", pattern="[^g]$", recursive=TRUE, include.dirs = TRUE, full.names = TRUE)
dirs

In [ ]:
# Get all image files: file names ending ".png" 
anger   = dir(grep("anger",   dirs, value = TRUE), pattern = "png$", full.names = TRUE)
disgust = dir(grep("disgust", dirs, value = TRUE), pattern = "png$", full.names = TRUE)
happy   = dir(grep("happy",   dirs, value = TRUE), pattern = "png$", full.names = TRUE)
sad     = dir(grep("sad",     dirs, value = TRUE), pattern = "png$", full.names = TRUE)
test_im = dir(grep("test",    dirs, value = TRUE), pattern = "png$", full.names = TRUE)

str(anger)
str(disgust)
str(happy)
str(sad)
str(test_im)

In [ ]:
ok = file.copy(  happy[60], "happy.png", overwrite = TRUE)
ok = file.copy(    sad[61],   "sad.png", overwrite = TRUE)
ok = file.copy(  anger[61], "anger.png", overwrite = TRUE)
ok = file.copy(disgust[61], "disgust.png", overwrite = TRUE)
IRdisplay::display_html('<img src="happy.png" width="200" style="float:left" /><img src="sad.png" width="200" style="float:left" /><img src="anger.png" width="200" style="float:left" /><img src="disgust.png" width="200" style="float:left" />')

Clearly the first is a happy face, but is the second a sad face, an angry face, or both?

<div style=color:darkblue;background-color:#fafaff;min-height:8em; >


<br>
<em> The second one seems like a sad face, third is angry, and the last one is disgust.
    One of the characteristics of sad face is having the inner corners of eyebrows raised, eyelids loose, and lip corners pulled down. Thus, we cna conlcude the first one to be the sad face. For the third picture, we can see the eyebrows being pulled down, lower eyelids pulled up,and lips tightened. This can show that she is angry. For the last one, the characteristic of wrinkled nose shows that she is closer to disgust than other emotions.</em>
<br>
<br>
    
<!-- Use Markdown or HTML to format your answer -->
    

</div>

--------------------------------


When working with image data, you often have many more Gigabytes of raw data than you have RAM memory available. Therefore, it is often not possible to work with all data "in memory". Resizing images often helps, but may cause loss of information.

The images for this competition are

- gray scale, so we need only one *color channel* 
- are only 48 by 48 pixels

Furthermore there are only 2538 pictures in the training set. Therefore, we are lucky enough to be able to retain all images in RAM, and don't have to do "special stuff" to handle reading in image files while fitting a model.

Reading in images pixelwise is easiest: We simply store each image as a long vector of pixel intensities, row by row. Also we will need a vector that contains the emotion label for each of the images.

In [ ]:
# Combine all filenames into a single vector
train_image_files = c(anger, happy, sad, disgust)

# Read in the images as pixel values (discarding color channels)
X_train = sapply(train_image_files, function(nm) c(readPNG(nm)[,,1])) %>% t() 
y = c(rep("anger", length(anger)), rep("happy", length(happy)), rep("sad", length(sad)), rep("disgust", length(disgust)))

X_test = sapply(test_im, function(nm) c(readPNG(nm)[,,1])) %>% t() 


# Change row and column names of X to something more managable
rownames(X_train)      = gsub(".+train/", "", rownames(X_train))
rownames(X_test) = gsub(".+test/",  "", rownames(X_test))

colnames(X_train) = colnames(X_test) = paste("p",1:ncol(X_train), sep="")

# Check result (are X_train, X_test, and y what we expect)
X_train[1:6,20:23] %>% print
table(y)
                
X_test[1:6,20:23] %>% print

In [ ]:
# combine X_train and X_test for the full data
X_combined <- rbind(X_train, X_test)

In [ ]:
# Visualization utility function
as_image = function(x, nr=sqrt(length(x))) {opar=par(mar=rep(0,4)); on.exit(par(opar)); image(t(matrix(x,nr))[,nr:1], col = gray(0:255/255),axes=F)}


options(repr.plot.width=4, repr.plot.height=4)
as_image(X_train[13,])
as_image(X_test[13,])

--------------------------------

# 3. Feature extraction
## 3.1. Spectral Features from Histogram

**DATA PREPARATION:**

Changing data frame to one long tibble data frame for combining the computed spectral features:

In [ ]:
data_prep <- as_tibble(X_combined, rownames = "id") %>%
    select(id, everything())
combined_data <- data_prep %>%
    pivot_longer(-id, names_to = "p_pos", values_to = "p_val")
combined_data %>%
    head()

**FEATURE EXPLANATION**

(Most of them from competition 2):
We added the following features:
* **_Level_**: It simply indicate the mean across the samples of a signal. It can inform the changes over time by detecting the dynamics in mean signals.
* **_Power_**: It represents the average squared amplitude of the signal. Power reflects the loudness of the signal, indicating the deviation from an absolute zero of X. The loudness may differ in different activities, as the strength of each activty differs. 
* **_Energy_**: It is the total power of the n samples. It may aid on distingushing between activities as the power of the signals all differ per activity. 
* **_Entropy_**: It interprets the average surprise of an observed value from histogram. The differences show by how low or high the probabililties are: the higher the probability, the less 'surprised' you are. 

* Based on characteristics of histogram, we extracted statistical features as well; giving information about their location of the distribution and width of the distribution:
    - _Mean_, _Standard Deivations_, _Quantiles_, _Skewness_, _Median_, _Mode_, _Maximum and Minimum_, _Kurtosis_
    - _Standard Error_, _Root Mean Squared_

* Based on the frequency of the band, we used signal spectruem to compare different facial expressions:
     - **_Mean frequency_**: We used general mean frequency to find which facial expression has higher mean frequency and what not. 
     - **_Spectral peak, spectral mean frequency, spectral standard deviation, spectral entropy_**:
    - The spectrum of a signal shows which frequencies a signal is composed of. It shows the number of fluctuations, including signals like a walking patterns ... etc. Peaks show the strength in which peaks are presented. Notice that height of each frequency is the square of a linear regression coefficient, meaning we can use to predict the signals from a wave. These spectral features can be used as a probability density function as well (ex, mean frequency). 
    
* **_Convolutions_**: For pattern recognition we used convolution. Convolution is a filter that passes over for example an image and extracting features that show a commonolatity in the images such that if the image has certain features, it belongs to a particular class. Thus, aiding in detecting facial expression. 
* **_Coefficient variance_**: It is the spread of the values in the epoch. The differences in values can be an indication of which signals in each epoch are different per facial expressions. 
* **_Width/heigth ratio_**: It is the ratio of width and height of the wave. This is an unstandardized method, which we aim to learn the differences in proportion of width and heigh per facial expression. 
* **_Magnitude Pixel Area_**: It is the absolute sum of the pixels. The sum of each pixels can aid differentiating different facial expressions, as the absolute sums are all different.

**FUNCTION CREATION:**

Features function are created to aid the calculation simplification. Below are the functions listed in the order of time domain, statistical, frequency, and other features.

In [ ]:
# Helper functions
most_common_value = function(x) {
    counts = table(x, useNA='no')
    most_frequent = which.max(counts)
    return(names(most_frequent))
}

# Level function
lev_fun <- function(x){
    lev = (1/length(x)) * sum(x)
    return(lev)
}

# Energy function
energy_fun = function(x) {
    ener = sum((x - mean(x))^2) 
    return(ener)
}

# Entropy function
ent_fun  <- function(x, nbreaks = nclass.Sturges(x)) {
    r = range(x)
    x_binned = findInterval(x, seq(r[1], r[2], len= nbreaks))
    h = tabulate(x_binned, nbins = nbreaks) # fast histogram
    p = h/sum(h)
    -sum(p[p>0] * log(p[p>0]))
}

# Interquartile Range function
iqr_fun <- function(x) {
    iqr = quantile(x, .75) - quantile(x, .25)
    return(iqr)
}

# Mode function
mode_fun <- function(x) {
    mode = as.integer(which.max(table(x)))
    return(mode)   
}

# Standard Error function
se_fun <- function(x) {
    se = sd(x) / sqrt(length(x))
    return(se)
}

# Root Mean Square function
rms_fun <- function(x){ #only on vectors
    rms = sqrt((1/length(x)*(sum(x^2))))
    return (rms)
}

# Mean Frequency function
mean_freq <- function(x) {
    out_1 = numeric()
    for (i in 1:length(x)){
        out_1[i] = i * x[i] }
        return(sum(out_1)/sum(x))
}

#Spectral Peak function
spec_peak = function(x) {
    spec = spectrum(x, log = 'n', plot = FALSE)
    peak = spec$freq[which.max(spec$spec)]
    return(peak)
}

# Spectral Mean Frequency function
spec_mean_freq <- function(x) {
    spec = spectrum(x, log = 'n', plot = FALSE)
    df = spec$freq[2] - spec$freq[1]
    sbar = sum(spec$freq * spec$spec * df)
    return(sbar)
}

# Sepctral Standard Deviation function
spec_sd <- function(x) {
    spec = spectrum(x, log = 'n', plot = FALSE)
    df = spec$freq[2] - spec$freq[1]
    svar = sum((spec$freq - mean(x))^2 * spec$spec * df)
    return(svar)
}

# Spectral Entropy function
spec_ent <- function(x) {
  spec = spectrum(x, log='n', plot = FALSE)$spec
  ent_fun(spec)
}

# Magnitude Pixel Area
mpa_fun <- function(s){
    mpa <- sum(abs(s))
    return(mpa)
}

# Pattern detection by convolutional neural networks function
conv_fun <- function(x, m) {
    w = 1/(2*m+1)
    conv = max(stats::filter(x, w, method = 'convolution'))
    return(conv)
}

**APPLICATION OF FUNCTIONS:**

Create a data frame for all the spectral features

In [ ]:
specf_combined <- function(values) {
    values %>%
        group_by(id) %>%
        summarize(
            # Level
            level_val = lev_fun(p_val),
            
            # Power
            power_val = mean(p_val^2),
            
            # Energy
            energy_val = energy_fun(p_val),
            
            # Entropy
            ent_val = ent_fun(p_val),
            
            # Mean
            mean_val = mean(p_val),
            
            # Standard deviation
            sd_val = sd(p_val),
            
            # Quantiles
            q_25_val = quantile(p_val, .25),
            q_75_val = quantile(p_val, .75),
            
            # Median absolute deviation
            mad_val = mad(p_val),
            
            # Mode
            mod_val = mode_fun(p_val),
            
            # Min and Max
            max_val = max(p_val),
            mix_val = min(p_val),
            
            # Kurtosis
            kur_val = e1071::kurtosis(p_val),
            
            # Standard Error
            se_val = se_fun(p_val),
            
            # Root Mean Square
            rms_val = rms_fun(p_val),
            
            # Coefficient variance
            coefvar_val = sd_val/mean_val,
            
            # Mean Frequency
            mfreq_val = mean_freq(p_val),
            
            # Spectral functions
            peak_val = spec_peak(p_val),
            s_meanfreq_val = spec_mean_freq(p_val),
            s_sd_val = spec_sd(p_val),
            s_ent_val = spec_ent(p_val),
            
            # Magnitude Pixel Area
            mpa_val = mpa_fun(p_val),
            
            # Convolutional Network
            conv_val = conv_fun(p_val, length(id)),
            
            # Witdh - Height Ratio
            whratio_val = p_val[which.max(p_val)]/length(id),
            
            # Difference in max and min of values
            diff_pv_val = max(p_val) - min(p_val)                 
        )
}

# Apply spectral features to combined data
spec_df <- specf_combined(combined_data)

Let's print out to see how the data frame looks like:

In [ ]:
spec_df %>%
    head()

## 3.2. Frey-Slate Features

In [ ]:
options(repr.plot.width=4*4, repr.plot.height=4)

# Compute edges by differencing neighboring pixels
im = matrix(X_train[756,],48)
h_edge = im[-1,] - im[-48,] # horizontal
v_edge = im[,-1] - im[,-48] # vertical
d_edge = h_edge[,-1] - h_edge[,-48] # diagonal

# Specify a threshold (hand tuned here on visual result)
threshold = .0625 

layout(t(1:4))
as_image(im)
as_image(h_edge < threshold,   47); mtext("horizontal edge pixels")
as_image(v_edge < threshold,   48); mtext("vertical edge pixels")
as_image(d_edge < threshold/2, 47); mtext("diagonal edge pixels")
# as_image((h_edge[,-1] < 0.1) & (v_edge[-1,] < 0.1), 47); mtext("edge pixels")

In [ ]:
# Load FreySlateFeatures function 
source("https://bit.ly/32um24j")

FreySlateFeatures(h_edge < threshold)

Now the edges and frey-slate features are loaded, we can create empty matrices to store frey slate features

In [ ]:
# Create three empty matrices to store the 16 Frey & Slate features
freyslate_h <- matrix(nrow = nrow(X_combined), ncol = 16)
freyslate_v <- matrix(nrow = nrow(X_combined), ncol = 16)
freyslate_d <- matrix(nrow = nrow(X_combined), ncol = 16)

We created the three matrices, but they lack column names, so we'll have to add them manually.

In [ ]:
# Right now there are no column names, we have to make sure they are correct
colnames(freyslate_h)[1:16] <- paste0(names(FreySlateFeatures(h_edge < threshold)), "_h")
colnames(freyslate_v)[1:16] <- paste0(names(FreySlateFeatures(v_edge < threshold)), "_v")
colnames(freyslate_d)[1:16] <- paste0(names(FreySlateFeatures(d_edge < threshold)), "_d")

head(freyslate_h)

Good. Now that the matrices are ready, we can construct and insert the features:

In [ ]:
# Create a for loop to construct the 16 Frey & Slate features
threshold = .0625 

for (i in 1:nrow(X_combined)) {
    
    im = matrix(X_combined[i,],48)
    
    h_edge = im[-1,] - im[-48,]
    v_edge = im[,-1] - im[,-48]
    d_edge = h_edge[,-1] - h_edge[,-48]
    
    # Frey & Slate features, 16 per orientation (horizontal, vertical and diagonal)
    freyslate_h[i,] <- FreySlateFeatures(h_edge < threshold)
    freyslate_v[i,] <- FreySlateFeatures(v_edge < threshold)
    freyslate_d[i,] <- FreySlateFeatures(d_edge < threshold)
    
}

head(freyslate_h)

Looks good, but some features have the exact same values per picture, which is a problem they hold little information. We will address this in the data cleaning section. For now, let's merge the two matrices into a single dataframe so that they're ready for further steps. We also need to add an "id" column for merging later.

In [ ]:
# Use cbind() to merge the two matrices
freyslate_features <- cbind(freyslate_h,
                            freyslate_v,
                            freyslate_d)

# An "id" column is still missing, which is needed to merge multiple features dataframes
X2 <- X_combined %>% as_tibble(rownames = "id")
X3 <- X2[,1]
freyslate_df <- cbind(X3, freyslate_features)

freyslate_df %>%
    head()

## 3.3. Histogram features from edges using Frey-Slate Functions
Histogram features work best on edges. An edge is a rapid change in pixel intensities, so if we compute the difference between two consecutive pixels, and check if it is larger than a certain threshold, we can find the pixels that are at the edge of an abrupt intensity change. We can use horizontal and vertical edges respectively. By computing differences in both north and west direction consecutively we filter out pixels that are part of a diagonal edge. 
Next up, we will create 4 different Histogram features, they are the mean, skewness, kurtosis and standard deviation. After that that we will create 16 different Frey & Slate features.

You can use the detected edge pixels to compute Frey and Slate type features: Histogram descriptives of the x and y locations of the 'on' pixels in the edge pixel maps.

In [ ]:
# Create a for loop to construct the Histogram features
threshold = .0625 

mean_horizontal <- mean_vertical <- mean_diagonal <- numeric(nrow(X_combined))

skew_horizontal <- skew_vertical <- skew_diagonal <- numeric(nrow(X_combined))

kurt_horizontal <- kurt_vertical <- kurt_diagonal <- numeric(nrow(X_combined))

sd_horizontal <- sd_vertical <- sd_diagonal <- numeric(nrow(X_combined))

sum_horizontal <- sum_vertical <- sum_diagonal <- numeric(nrow(X_combined))

lev_horizontal <- lev_vertical <- lev_diagonal <- numeric(nrow(X_combined))

energy_horizontal <- energy_vertical <- energy_diagonal <- numeric(nrow(X_combined))

spec_peak_horizontal <- spec_peak_vertical <- spec_peak_diagonal <- numeric(nrow(X_combined))

spec_meanfreq_horizontal <- spec_meanfreq_vertical <- spec_meanfreq_diagonal <- numeric(nrow(X_combined))

spec_sd_horizontal <- spec_sd_vertical <- spec_sd_diagonal <- numeric(nrow(X_combined))

spec_ent_horizontal <- spec_ent_vertical <- spec_ent_diagonal <- numeric(nrow(X_combined))

mpa_horizontal <- mpa_vertical <- mpa_diagonal <- numeric(nrow(X_combined))

for (i in 1:nrow(X_combined)) {
    
    im = matrix(X_combined[i,],48)
    
    h_edge = im[-1,] - im[-48,]
    v_edge = im[,-1] - im[,-48]
    d_edge = h_edge[,-1] - h_edge[,-48]
    
    # Mean
    mean_horizontal[i] <- mean(h_edge < threshold)
    mean_vertical[i] <- mean(v_edge < threshold)
    mean_diagonal[i] <- mean(d_edge < threshold)
    
    # Skewness
    skew_horizontal[i] <- e1071::skewness(h_edge < threshold)
    skew_vertical[i] <- e1071::skewness(v_edge < threshold)
    skew_diagonal[i] <- e1071::skewness(d_edge < threshold)

    # Kurtosis
    kurt_horizontal[i] <- e1071::kurtosis(h_edge < threshold)
    kurt_vertical[i] <- e1071::kurtosis(v_edge < threshold)
    kurt_diagonal[i] <- e1071::kurtosis(d_edge < threshold)

    # Standard deviation
    sd_horizontal[i] <- sd(h_edge < threshold)
    sd_vertical[i] <- sd(v_edge < threshold)
    sd_diagonal[i] <- sd(d_edge < threshold)
    
    # Sum
    sum_horizontal[i] <- sum(h_edge < threshold)
    sum_vertical[i] <- sum(v_edge < threshold)
    sum_diagonal[i] <- sum(d_edge < threshold)

    # Level
    lev_horizontal[i] <- lev_fun(h_edge < threshold)
    lev_vertical[i] <- lev_fun(v_edge < threshold)
    lev_diagonal[i] <- lev_fun(d_edge < threshold)

    # Energy
    energy_horizontal[i] <- energy_fun(h_edge < threshold)
    energy_vertical[i] <- energy_fun(v_edge < threshold)
    energy_diagonal[i] <- energy_fun(d_edge < threshold)

    # Sepctral peak
    spec_peak_horizontal[i] <- spec_peak(h_edge < threshold)
    spec_peak_vertical[i] <- spec_peak(v_edge < threshold)
    spec_peak_diagonal[i] <- spec_peak(d_edge < threshold)

    # Spectral Mean Frequency
    spec_meanfreq_horizontal[i] <- spec_mean_freq(h_edge < threshold)
    spec_meanfreq_vertical[i] <- spec_mean_freq(v_edge < threshold)
    spec_meanfreq_diagonal[i] <- spec_mean_freq(d_edge < threshold)

    # Spectral Standard Deviations
    spec_sd_horizontal[i] <- spec_sd(h_edge < threshold)
    spec_sd_vertical[i] <- spec_sd(v_edge < threshold)
    spec_sd_diagonal[i] <- spec_sd(d_edge < threshold)

    # Spectral Entropy
    spec_ent_horizontal[i] <- spec_ent(h_edge < threshold)
    spec_ent_vertical[i] <- spec_ent(v_edge < threshold)
    spec_ent_diagonal[i] <- spec_ent(d_edge < threshold)

    # Magnitude Pixel Area
    mpa_horizontal[i] <- mpa_fun(h_edge < threshold)
    mpa_vertical[i] <- mpa_fun(v_edge < threshold)
    mpa_diagonal[i] <- mpa_fun(d_edge < threshold)
    
}

# Combine all of the features into one data frame
histogram_df <- cbind(mean_horizontal, mean_vertical, mean_diagonal,
                      skew_horizontal, skew_vertical, skew_diagonal,
                      kurt_horizontal, kurt_vertical, kurt_diagonal,
                      sd_horizontal, sd_vertical, sd_diagonal,
                     sum_horizontal, sum_vertical, sum_diagonal,
                     lev_horizontal, lev_vertical, lev_diagonal,
                     energy_horizontal, energy_vertical, energy_diagonal,
                     spec_peak_horizontal, spec_peak_vertical, spec_peak_diagonal,
                     spec_meanfreq_horizontal, spec_meanfreq_vertical, spec_meanfreq_diagonal,
                     spec_sd_horizontal, spec_sd_vertical, spec_sd_diagonal,
                     spec_ent_horizontal, spec_ent_vertical, spec_ent_diagonal,
                     mpa_horizontal, mpa_vertical, mpa_diagonal) %>%
                as.data.frame()

Looks great, last step is to add an id column for merging with other frames later on.

In [ ]:
# An "id" column is still missing, which is needed to merge multiple features dataframes. Make sure to only run this chunk once.
X2 <- X_combined %>% as_tibble(rownames = "id")
X3 <- X2[,1]

histogram_df <- cbind(X3, histogram_df)

In [ ]:
# Let's take a look
histogram_df %>%
    head()

## 3.4. Histogram of Oriented Gradients (HOG) features

This feature was adapted from group 1 and group 5, who were inspired by students from [last competition](https://www.kaggle.com/code/thomassvisser/can-we-predict-the-right-facial-expression-round-2/notebook?scriptVersionId=77717791).

According to this [website](https://towardsdatascience.com/hog-histogram-of-oriented-gradients-67ecd887675f), the `histogram of oriented gradients (HOG)` is used in image processing to detect objects. These features count occurrences of gradient orientation in the localized portion of an image, by focusing on structuure or the shape of the object. It uses magnitude and angles of the gradient to compute the features, which will contribute to increasing accuracy of our model.

### Step 1: 
Preprocessing. In this step, we would convert each img a 48 x 48 matrix to calculate gradient.

In [ ]:
to_matrix = function(image_matrices, col = 48, row = 48) {
  
  image_pixels = list()
  
  for(i in 1:nrow(image_matrices)) {
    
    # extract pixel matrix for each image
    image_pixels[[i]] = image_matrices[i, ] %>% matrix(ncol = col, nrow = row) 
  }
  return(image_pixels)
}

dim(to_matrix(X_combined)[[1]])

### Step 2:
Calculate the Gradient Images. To calculate a HOG descriptor, we need to first calculate the horizontal and vertical gradient. Then, we can calculate magnitude and direction of gradient by following formula, in which `g` represents magnitude, and `θ` represents direction in degrees. Magnitude indecates the intensity of the direction change and direction indicates the angle of the change at a point.

In [ ]:
# calculate horizontal and vertical gradients
get_gradients <- function(image_matrices, size = 48) {
    add_zeros_column = matrix(rep(0, size), ncol = 1)
    add_zeros_row = matrix(rep(0, size), nrow = 1)
    h_gradient = v_gradient = list()
  
    for(i in 1:length(image_matrices)) {
    
        removed_column <- image_matrices[[i]][,-size]
        h_gradient[[i]] <- image_matrices[[i]] - cbind(add_zeros_column, removed_column)
        
        removed_row <- image_matrices[[i]][-1, ]
        v_gradient[[i]] <- image_matrices[[i]] - rbind(removed_row, add_zeros_row)
    
    }
    return(list(h = h_gradient, v = v_gradient))
}

### Step 3:
Calculate Histogram of Gradients in cells. Our facial data has all pictures in 48 × 48 pixels, so we choose to calculate histograms for every 4 × 4 pixels cell, because they can well capture the shape of faces. Thus, we had 12 × 12 cells for each image. We used 9 bins to divide the whole range of direction (0: 180), and each pixels in a cell would be weighted by magnitue and allocated to two adjacent bins based on the direction position in the scale.

In [ ]:
# calculate magnitude and direction of gradient
get_magnitude <- function(h_gradient, v_gradient) {
    magnitude <- list()
  
    for(i in 1:length(h_gradient)) {
        magnitude[[i]] <- sqrt(h_gradient[[i]]^2 + v_gradient[[i]]^2)
    }
    return(magnitude)
}

### Step 4:
Block Normalization. In the previous step, we calculate the histogram by "counting" and "weighting" the direction, so the histogram would be influenced by "counting" numbers a lot. Thus, we would normalize the histograms to specify the change on edges and corners. To do so, we chose to normalize the cells on larger 8 × 8 pixels blocks, which contain 4 cells each, and there are 11 × 11 blocks for each image. To make the vector as one-dimension, we also concatenated the each block to a 36 × 1 vector, as in total, 36 × 11 × 11 = 4356.

In [ ]:
get_direction <- function(h_gradient, v_gradient) {
    direction <- list()
    for(i in 1:length(h_gradient)) {
        
        direction[[i]] <- (atan(v_gradient[[i]] / h_gradient[[i]]))* (180/pi)
        
        # replace NAs with 0
        direction[[i]][is.na(direction[[i]])] <- 0
        
        index <- direction[[i]] < 0
        direction[[i]][index] <- direction[[i]][index] + 180
    }
    return(direction)
}

### Step 5:
Calculate the Histogram of Oriented Gradients feature vector. After calculating an image with all steps above, we made a function to loop over all images to calculate the whole feature vector.

In [ ]:
# a function to calculate his of a img from direction and magnitude matrics:
get_block_hist <- function(direction, magnitude){
    
    upper_limit <- seq(20, 160, 20)
    lower_limit <- seq(0, 140, 20)
    column <- list()
    row_blocks <- list()
  
    # separate image into blocks
    pixels <- seq(0, 44, by = 4)
  
    # loop over columns in 4*4 blocks
    for(c in pixels){
    
    # loop over rows in 4*4 blocks
    for(r in pixels){ 
        
        # select a cell
        direction_block <- direction[(r + 1):(r + 4), (c + 1):(c + 4)]
        magnitude_block <- magnitude[(r + 1):(r + 4), (c + 1):(c + 4)]
      
        # because the intervals are closed to the right the last break point 
        # is set 
        # manually so that values of 180 are included
        bin_index <- cut(direction_block, breaks = c(seq(0, 160, 20), 181), 
                        right = FALSE, labels = FALSE)
        bin_index_matrix <- matrix(bin_index, 4, 4)
        
        # the bins are emptied for every matrix
        bin <- numeric()
      
        for(j in 1:9){
            
            # special case bin 9
            if(j == 9){
          
                # sum(magnitude_vector * ((180 - direction_vector) / 20))
                bin[j] <- sum(c(magnitude_block[bin_index_matrix == j]) * 
                             ((180 - c(direction_block[bin_index_matrix == j])) / 20))
          
                # bin_content + sum(magnitude_vector * ((direction_vector - 0) / 20))
                bin[1] <- bin[1] + sum(c(magnitude_block[bin_index_matrix == j]) * 
                                      ((c(direction_block[bin_index_matrix == j]) - 0)/20))# 0 or 160?
                
                # the last step for each bin, store the sum of the bin in colunm
                row_blocks[[which(pixels == r)]] <- bin
                column[[which(pixels == c)]] <- row_blocks
          
            }
            
            # sum(magnitude_vector * ((upper_limit - direction_vector)/20))
            bin[j] <- sum(c(magnitude_block[bin_index_matrix == j]) * 
                         ((upper_limit[j] - c(direction_block[bin_index_matrix == j])) / 20))
            
            # sum(magnitude_vector * ((direction_vector - lower_limit)/20))
            bin[j + 1] <- sum(c(magnitude_block[bin_index_matrix == j]) * 
                             ((c(direction_block[bin_index_matrix == j]) - lower_limit[j]) / 20))
        }
    }
    }
    return(column)
}

In [ ]:
# 8*8 Block Normalization
# we have 12 blocks in a row so 11 big blocks for normalization when we use double pixels as row and col
concatenate_block_HOGs <- function(blocks) {
  
    concatenated_columns <- list()
    concatenated_all <- list()
  
    # loop over column
    for(i in 1:11) {
    
        # loop over row
        for(j in 1:11){
      
            vect <- unlist(as.vector(c(blocks[[i]][j], blocks[[i]][j + 1],
                                                           blocks[[i + 1]][j], blocks[[i + 1]][j + 1])))
            # Normalized vector
            concatenated_columns[[j]] <- vect/sum(sqrt(vect^2))
        }
        concatenated_all[[i]] <- concatenated_columns
    }
    # 9*4*11*11 = 4356
    HoG_matrix <- matrix(unlist(concatenated_all), 1, 4356)
    return(HoG_matrix)
}

In [ ]:
# Calculate the Histogram of Oriented Gradients feature vector
# Calculate all img features
get_all_hog <- function(img){
    
    HOGs_feature_matrix <- matrix(0, nrow(img), 4356)
  
    # get all image matrices in the right format
    image_matrices <- to_matrix(img)
  
    # get h & v gradients
    h_gradient <- get_gradients(image_matrices)$h
    v_gradient <- get_gradients(image_matrices)$v
  
    img_dir <- get_direction(h_gradient, v_gradient)
    img_mag <- get_magnitude(h_gradient, v_gradient)
    
    for(i in 1:nrow(img)) { 
    
    HOGs <- get_block_hist(img_dir[[i]], img_mag[[i]])
    
    HOGs_feature_matrix[i,] <- concatenate_block_HOGs(HOGs)

  }
    colnames(HOGs_feature_matrix) <- paste("HOG", 1:4356)
    return(HOGs_feature_matrix)
}

Apply X_combined to the function to extract HOG_features:

In [ ]:
# Extract
HOG_features <- get_all_hog(X_combined)
rownames(HOG_features) <- rownames(X_combined)

Change to tibble format and add id as a column:

In [ ]:
HOG_features2 <- as_tibble(HOG_features)
HOG_features2 <- rownames_to_column(HOG_features2, "id")
HOG_features2 %>%
    head()

# 4. Data preparation 
### Combining and Cleaning
### 4.1. Data Frames Merging

Merge spectral features for the whole image, freyslate features, histogram spectral features for edge features, and histogram of oriented gradients features into one data frame.

In [ ]:
combined_df <- X_combined %>%
    as_tibble(rownames = "id") %>%
    left_join(spec_df, by = "id") %>%
    left_join(freyslate_df, by = "id") %>%
    left_join(histogram_df, by = "id") %>%
    left_join(HOG_features2, by = "id") %>%
    select(-id)

# Check result
combined_df[1:6,2348:2358] %>% print
dim(combined_df)

## 4.2. Data cleaning: 
Using obervation and caret package to remove redundant variables

* Variables that give NAs
* Variables with near zero variation
    - Have all same values or all NAs
    - Compare most frequent to second most frequent value
    - Compare number of unique values to number of observations
* Variables that are highly correlated
    - High correlation between variables provide similar or same information
    - May be an indicator of instability of coefficient estimates

In [ ]:
# replace NAs with 0
combined_df <- combined_df %>%
    replace(is.na(.), 0)

Before the further preprocessing, a `Principal Components Analysis (PCA)` is calculated over the whole set of features to get an idea of how many "meaningful dimensions" are among the predictors. 

In [ ]:
zv_PCA <- c(which(apply(combined_df, 2, var)==0))
zv_PCA
## Id and these columns need to be removed before calculating PCA because they are constant or have
## Zero Var, prevents prcomp to execute
df_PCA <- prcomp(combined_df[,-zv_PCA], scale = TRUE)
## How many Principal components have an eigenvalue > 1?
paste0("A PCA on the complete set of our predictors results in ", 
       length(which(df_PCA$sdev > 1)), " principal components")

Now, let's get to preprocessing:

In [ ]:
# Near zero variation
nearzeroval <- combined_df %>%
    caret::nearZeroVar(names = TRUE)

# Remove nearzeroval variables
combined_df_cleaned <- combined_df %>%
    select(-all_of(nearzeroval))

In [ ]:
# High correlations
cor_val <- combined_df_cleaned %>%
    cor() %>%
    replace(is.na(.), 0) # cor(0, 0) results in NAs so we replace them with 0
var <- caret::findCorrelation(cor_val, 0.99)

# Remove nearzeroval and var variables
combined_df_cleaned <- combined_df_cleaned %>%
    select(-all_of(var))

# print the dimensions
dimdf <- dim(combined_df_cleaned)
print(paste0("This is the current dimensions of the dataframe: ", dimdf[1], " x ", dimdf[2], sep = ''))

What can we learn from this:
* The number of remaining features after cleaning is very far away from the number of dimensions calculated with a principal component analysis. However, when being more strict with the correlation threshold, the models calculated in the following steps perform significantly worse when predicting test data. Thus, dimension reduction by means of a PCA is not advisable in this case

# 5. Splitting data for training & validation sets

First, we divide training and test data into different dataframes
* Two for full dataset
* Two for clean dataset after removing `near zero variance` and `high correlations`

In [ ]:
train_full_df <- combined_df[1:nrow(X_train), ]
test_full_df <- combined_df[-c(1:nrow(train_full_df)), ]

clean_train_df <- combined_df_cleaned[1:nrow(X_train), ]
clean_test_df <- combined_df_cleaned[-c(1:nrow(train_full_df)), ]

Check if the test and training datasets have same column lengths:

In [ ]:
dim_train_full <- dim(train_full_df)
dim_test_full <- dim(test_full_df)
print(paste0("This is the current column lengths of the full training dataset is: ", dim_train_full[2], " and for full test dataset is ", dim_test_full[2], sep = ''))

dim_clean_train <- dim(clean_train_df)
dim_clean_test <- dim(clean_test_df)
print(paste0("This is the current column lengths of the clean training dataset is: ", dim_clean_train[2], " and for clean test dataset is ", dim_clean_test[2], sep = ''))

Now we're going to divide the training df into 20% validation set and 80% training dataset, as it will yield more accurate results:

In [ ]:
trainind <- caret::createDataPartition(y, p = 0.8, list = FALSE)

# Train
# Use below for model fitting!
train_X <- clean_train_df[trainind, ]
train_Y <- y[trainind]

# Test
test_X <- clean_train_df[-trainind, ]
test_Y <- y[-trainind]

# 6. Model fitting

To figure out which model provides the best trade off between bias and variance, between accuracy and flexibility, one strategy is to fit both a flexible and a more rigid model and determine from CV error which direction on the flexiblity axis we should go to avoid overtraining.

In [ ]:
suppressMessages(require(caret))

## Use multiple cores whenever possible
library(parallel)
suppressMessages(library(doParallel))
(detectCores() - 1) %>% makeCluster() %>% registerDoParallel()

In [ ]:
# set traincontrol for cross-validation
trCntrl = trainControl('cv', 5, allowParallel = TRUE)

## 6.1. Linear Discriminant Analysis (LDA)

As our first model we tried Linear Discriminant Analysis (LDA).
LDA is a linear model used for classification and dimensionality reduction. First created for two classes in 1936 by R. Fisher and then generalized for multiple classes by C.R. Rao. It is used to solve various classification problems. 

A good example is given by the book: think of an online banking service trying to determine whether or not a transaction being performed is fraudelent or not, on the basis of the user's IP address, past transaction history, and so forth (James et al., 2013).  

In our own problem we also try to classify, but this time 4 classes and have many different features. For the LDA we assume that X is drawn from a multivariate Gaussian (or multivariate normal) distribution, with a class-specific mean vector and a common covariance matrix (James et al., 2013).

Unfortunately in our notebook LDA failed to run succesfully.

In [ ]:
# LDA does not function ! 
#set.seed(2020)
#fit_lda_f = train(train_X, train_Y, method='lda', trControl=trCntrl)
#fit_lda_f


## 6.2. Quadratic Discriminant Analysis (QDA)
The main difference between LDA and Quadratic Discriminant Analysis (QDA) is that QDA assumes that each class has its own covariance matrix, whereas LDA assumes that each class has the same covariance matrix.

QDA is a much more flexible classifier than LDA and has substantially higher variance. This could potentially lead to better prediction performance compared to LDA (James et al., 2013).

In general, QDA is recommended if the training set is very large. In our case, our training set is definitely large enough.

When trying to use QDA that was not pre-processed, it failed (without PCA or Ridge regularization).

In [ ]:
# QDA with PCA as preprocessing step
set.seed(2020)
fit_qda_f_pca = train(train_X, train_Y, method='qda', trControl=trCntrl, preProcess = "pca")
fit_qda_f_pca

*Since the accuracy is not as high for QDA (.82) as for the best performing models, ridge regularized QDA was not tested*

## 6.3. K-nearest neighbors (KNN)
K-nearest neighbors (KNN) is a type of supervised learning algorithm used for both regression and classification. KNN tries to predict the correct class for the test data by calculating the distance between the test data and all the training points. Then select the K number of points which is closet to the test data. The KNN algorithm calculates the probability of the test data belonging to the classes of ‘K’ training data and class holds the highest probability will be selected. In the case of regression, the value is the mean of the ‘K’ selected training points. Source: https://medium.com/swlh/k-nearest-neighbor-ca2593d7a3c4

In [ ]:
## KNN (unscaled)
set.seed(2020)
fit_knn = train(train_X, train_Y, method='knn', trControl=trCntrl)
plot(fit_knn)
fit_knn

*Highest accuracy within this model was reached with k = 7, however rather poor (.519)*

In [ ]:
## KNN (scaled)
set.seed(2020)
fit_knn_s = train(train_X, train_Y, method='knn', trControl=trCntrl, preProcess = "scale")
plot(fit_knn_s)
fit_knn_s

*Highest accuracy within this model was reached with k = 9, a good bit higher (.698) than
in unscaled knn*

## 6.4. The Lasso
Next up is the Lasso. The Lasso is a shrinkage method and a clear improvement over the Ridge regression. Ridge regression will always include all $p$ predictors in the final model. This is because the penalty in ridge regression will shrink all of the coefficients very very close to zero, but it will not actually set any of them to actual zero. This is an issue in model interpretation when the number of variables $p$ is quite large (James et al., 2013).

The Lasso uses a different penalty and shrinks all coefficient estimates to zero, however the $l1$ penalty forces some of the estimates down to exactly zero as the tuning parameter Lambda increases. This results in variable selection (James et al., 2013).

In [ ]:
# Fit a cross-validated Lasso
set.seed(2020)
fit_lasso <- glmnet::cv.glmnet(as.matrix(train_X), as.matrix(train_Y),
                      family = "multinomial",
                      type.measure = "class",
                      nfolds = 5,
                      parallel = TRUE,
                      alpha = 1)
fit_lasso

In [ ]:
# Predictions with the tuned LASSO-model on the training set
pred_lasso <- predict(fit_lasso, as.matrix(test_X), 
                          s = "lambda.min", 
                          type = 'class', 
                          alpha = 1) %>% 
                          as.factor()

caret::confusionMatrix(pred_lasso, factor(test_Y))

With the optimal lambda (0.01898), LASSO regression reached an accuracy of .78 on our pre defined test set

## 6.5. Ridge Regression
Ridge regression is an improvement over least squares as it uses a tuning parameter called Lambda. As Lambda increases, the flexibility of the ridge regression fit decreases, leading to decreased variance but increased bias (James et al., 2013).

Even though the Lasso is a general improvement to ridge regression, we want to include ridge regression to make sure we try out all possible models. 


In [ ]:
fit_ridge <- train(train_X, train_Y, 
                   method = 'glmnet', 
                   trControl = trCntrl, 
                   tuneGrid = expand.grid(alpha = 0, lambda = 0.002))
fit_ridge

Predict the fitted model:

In [ ]:
pred_ridge <- predict(fit_ridge, test_X, type ='raw') 
confusionMatrix(pred_ridge, factor(test_Y))

Ridge Regression with a constant lambda of 0.002 reached an accuracy of .83 on our pre defined test set

## 6.6. Classification tree
A classification tree is very similar to a regression tree, except that it is used to predict a qualitative response rather than a quantitative one. Re-call that for a regression tree, the predicted response for an observation is given by the mean response of the training observations that belong to the
same terminal node. 

In contrast, for a classification tree, we predict that each observation belongs to the most commonly occurring class of training observations in the region to which it belongs. In interpreting the results of a classification tree, we are often interested not only in the class prediction corresponding to a particular terminal node region, but also in the class proportions among the training observations that fall into that region (James et al., 2013).

In [ ]:
## Fit a CART using 5-fold cross-validation to tune the complexity parameter
## (fixed cP parameter was taken over from teachers example implemented in 
## the raw version of this notebook)
set.seed(2020) 
tt <- Sys.time()
fittree = train(x=train_X, y=train_Y, method='rpart', trControl = trCntrl,
                tuneGrid = data.frame(cp=.02))
fittree
(dur <- Sys.time() - tt)
fittree$results$Accuracy

In [ ]:
## Tuning the cP parameter
suppressWarnings({
set.seed(2020)
cps <- seq(0.001, 0.01, 0.001)
cp_acc <- as.matrix(cps, byrow = TRUE)
acc_s <- c()   
for (i in cps) {
    acc <- train(x=train_X, y=train_Y, method='rpart', trControl = trCntrl,
                    tuneGrid = data.frame(cp = i))$results$Accuracy
    acc_s <- c(acc_s, acc)
    
   
}
acc_s <- as.matrix(acc_s, byrow = TRUE)                    
tune <- cbind(cp_acc, acc_s)
print(tune)
})

Iterative usage of different cPs gave cP = 0.003 as best value with the best outcome in terms of 
accuracy

In [ ]:
## Fitting the CART again with optimized cP
set.seed(2020) 
fittree_tune = train(x=train_X, y=train_Y, method='rpart', trControl = trCntrl,
                     tuneGrid = data.frame(cp=0.003))
fittree_tune

*The optimized CART yielded an accuracy of .62*

In [ ]:
## Check performance on our defined test set
predtree = predict(fittree_tune, test_X, type='raw') 
confusionMatrix(predtree, factor(test_Y))

*Predictions accuracy on our predefined test set was a bit below (.59) the predictions on the training set*

## 6.7. Support Vector Machine:
The objective of the support vector machine algorithm is to find a hyperplane in an N-dimensional space(N — the number of features) that distinctly classifies the data points.

To separate the two classes of data points, there are many possible hyperplanes that could be chosen. Our objective is to find a plane that has the maximum margin, i.e the maximum distance between data points of both classes. Maximizing the margin distance provides some reinforcement so that future data points can be classified with more confidence.

Hyperplanes are decision boundaries that help classify the data points. Data points falling on either side of the hyperplane can be attributed to different classes.

Support vectors are data points that are closer to the hyperplane and influence the position and orientation of the hyperplane. Using these support vectors, we maximize the margin of the classifier. Deleting the support vectors will change the position of the hyperplane. These are the points that help us build our SVM. 

Source: https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47

In [ ]:
set.seed(2020)
tune_svm = e1071::tune.svm(y ~., 
                       data = data.frame(train_X, y = as.factor(train_Y)), 
                       kernel = 'radial',
                       gama = c(0.5,1,2,3,4),
                       range = list(cost = c(0.001,0.01,0.1,1,5,10, 100)))

tune_svm$best.model

**We choose cost = 1 because it allows us to use the most accurate/ best model**

In [ ]:
set.seed(2020)
fit_svm = e1071::svm(y~., data = data.frame(train_X, y = factor(train_Y)), cost = 1, 
                scale = TRUE, kernel = 'radial')

pred_svm = predict(fit_svm, test_X, type = 'raw')
caret::confusionMatrix(pred_svm, factor(test_Y))

Predictive accuracy of the optimal SVM does quite well, .905 accuracy on our predefined test data, **the best accuracy score so far**.

## 6.8. Random Forests
Bagging involves creating multiple copies of the original train-
ing data set using the bootstrap, fitting a separate decision tree to each copy, and then combining all of the trees in order to create a single predictive model. Random forests provide an improvement over bagged trees by way of a small tweak that decorrelates the trees. As in bagging, we build a number of decision trees on bootstrapped training samples. But when building these decision trees, each time a split in a tree is considered, a random sample of $m$ predictors is chosen as split candidates from the full set of $p$ predictors. The split is allowed to use only one of those $m$ predictors. A fresh sample of
$m$ predictors is taken at each split (James et al., 2013).

Random forests are probably the least susceptible to overtraining and is considered one of the best "off the shelf" machine learning algorithms in the sense that they require little expertise in application, and easily perform well without tuning. 

In [ ]:
## Fitting a random forest (Without auto tuning the meta parameters)
set.seed(2020)
fit_rf = caret::train(train_X, train_Y, method='ranger', 
    trControl = trCntrl, 
    tuneGrid = data.frame(mtry=9, splitrule="gini", min.node.size=10)
)
fit_rf$finalModel

In [ ]:
predrf = predict(fit_rf, test_X) 
confusionMatrix(predrf, factor(test_Y))

Predictive accuracy of the random forest was .875 with no tuning, however further tuning might yield even better performance

In [ ]:
## Tuning of the grid parameters (number of trees kept constant)
## (Different intermediate steps of altering mtry and min.node.size were omitted here, 
## mtry was tested from values 1 to 12, min node size was tested from
## values 1 to 20)

tgrid <- expand.grid(
  .mtry = 12,
  .splitrule = "gini",
  .min.node.size = c(1, 2, 5))

fit_rf_tune1 = caret::train(train_X, train_Y, method='ranger', 
    trControl = trCntrl, 
    tuneGrid = tgrid)

fit_rf_tune1$finalModel

Keeping the number of trees constant, the optimal random forest seems to
arise from mtry = 12 and minimal node size 1

In [ ]:
predrf_tune1 = predict(fit_rf_tune1, test_X) 
confusionMatrix(predrf_tune1, factor(test_Y))

With constant number of trees and optimal mtry and min node size, 
an accuracy of .88 is reached


In [ ]:
## Fitting optimized random forest:
## Tuning the number of trees
suppressWarnings({
set.seed(2020)
num_trees <- seq(500, 2000, 100)
trees_acc <- as.matrix(num_trees, byrow = TRUE)
acc_t <- c()   
for (i in num_trees) {
    oob_tree <- train(train_X, train_Y, method='ranger', 
    trControl = trCntrl, 
    tuneGrid = data.frame(mtry=12, splitrule="gini", min.node.size=1),
    num.trees = i)$finalModel$prediction.error
    acc_t <- c(acc_t, oob_tree)
}
acc_t <- as.matrix(acc_t, byrow = TRUE)                    
tuned_trees <- cbind(trees_acc, acc_t)
print(tuned_trees)
})

Lowest OOB error in the range of inspected number of trees arises from 1900 trees, however result is likely to vary with new run since the values are very close to each other

In [ ]:
set.seed(2020)
fit_rf_king = train(train_X, train_Y, method='ranger', 
    trControl = trCntrl, 
    tuneGrid = data.frame(mtry=12, splitrule="gini", min.node.size=1), 
    num.trees = 1900)

fit_rf_king$finalModel

In [ ]:
predrf_tune_king = predict(fit_rf_king, test_X) 
confusionMatrix(predrf_tune_king, factor(test_Y))

Predictive accuracy on the test set did not increase compared with the 500 trees, still around .88

## 6.9. eXtreme Gradient Boosted Tree
Boosting is quite similar to bagging. In bagging, each tree is built on a bootstrap data set, independent of the other trees. 

Boosting works in a similar way, except that the trees are
grown sequentially: each tree is grown using information from previously grown trees. Boosting does not involve bootstrap sampling; instead each tree is fit on a modified version of the original data set.

Unlike fitting a single large decision tree to the data, which amounts to fitting the data hard and potentially overfitting, the boosting approach instead *learns slowly* (James et al., 2013).

In [ ]:
## fitting a boosted tree (without changing the meta parameters)
set.seed(2020)
fit_xgb = train(train_X, train_Y, method="xgbTree", 
    trControl = trCntrl, 
    tuneGrid = data.frame(
        nrounds=600, max_depth=4, eta=.3, 
        gamma=0, colsample_bytree=0.95, 
        min_child_weight=1, subsample=1)
)
fit_xgb

In [ ]:
## Validation set estimate for xgb
pred_xgb = predict(fit_xgb, test_X) 
confusionMatrix(pred_xgb, factor(test_Y))

After very long calculation time, Extreme gradient boosting yields an accuracy of 
0.91 with all the indicated tuning parameters. In the submission on the test data, Extreme boosting performed slightly worse than Random Forest and SVM. Therefore, we decided against further tuning of this model

# 7. Model comparison & conclusion

In [ ]:
# accuracy scores for each model saved in vector

lasso_confusion <- confusionMatrix(pred_lasso, factor(test_Y))
ridge_confusion <- confusionMatrix(pred_ridge, factor(test_Y))
tree_confusion <- confusionMatrix(predtree, factor(test_Y))
svm_confusion <- confusionMatrix(pred_svm, factor(test_Y))
rf_confusion <- confusionMatrix(predrf, factor(test_Y))
rf_tune_confusion <- confusionMatrix(predrf_tune_king, factor(test_Y))
xgb_confusion <- confusionMatrix(pred_xgb, factor(test_Y))

In [ ]:
# credit to group 1

models <- rbind(lasso_confusion$overall, ridge_confusion$overall, tree_confusion$overall,
                svm_confusion$overall, 
                rf_confusion$overall, rf_tune_confusion$overall, xgb_confusion$overall) %>%
        as_tibble %>%
        add_column(Model = c("Lasso", "Ridge", "Classification Tree", "SVM", "Random Forest",
                             "T-Random Forest", "Boosted Tree"))

models %>%
    ggplot(aes(x = Model, y = Accuracy, fill = Model)) +
    geom_col() +
    # for getting the error bar
    geom_errorbar(aes(ymax = AccuracyLower, ymin = AccuracyUpper)) +
    theme_minimal() +
    theme(axis.text = element_text(size = 10), axis.title = element_text(size = 13),
          plot.title = element_text(size = 15, hjust = 0.5), legend.position = "none") +
    labs(title = "Accuracy Estimates for Each Model")

### References:

P. Lucey, J. F. Cohn, T. Kanade, J. Saragih, Z. Ambadar and I. Matthews, "The Extended Cohn-Kanade Dataset (CK+): A complete dataset for action unit and emotion-specified expression," *2010 IEEE Computer Society Conference on Computer Vision and Pattern Recognition - Workshops*, 2010, pp. 94-101. DOI: 10.1109/CVPRW.2010.5543262. 

Jack, R. E., Garrod, O. G. B., Yu, H., Caldara, R., & Schyns, P. G. (2012). *Facial expressions of emotion are not culturally universal. Proceedings of the National Academy of Sciences*, *109*(19), 7241–7244. DOI: 10.1073/pnas.1200155109

James, G., Witten, D., Hastie, T., & Tibshirani, R. (2013). An Introduction to Statistical Learning: with Applications in R. Springer Publishing.

## Conclusion:

From the accuracy plots, we can observe that the boosted tree yields the highest accuracy. However, considering that boosted tree is more prone to overfitting than random forests, we chose random forests as our final model to fit. The sensitivity and specificity of random forests also look quite proper, showing it is appropriate to choose this as our final model.

# 8. Formatting your submission file

To format your submission file, you can use the following code:

In [ ]:
## Make predictions
pred_svm = predict(fit_svm, clean_test_df, type='raw')

## Write to file
tibble(file = rownames(X_test), category = pred_svm) %>% 
    write_csv(path = "submission.csv")

## Check result
cat(readLines("submission.csv",n=20), sep="\n")